## How to transfer your model from BluePyOpt to Basal Ganglia Data (using Snudda format)

### Structure of the BluePyOpt model 

The model optimisation should be a folder, containing the follow files and subdirectories:

    model/
        config/
                parameters.json
                mechanisms.json
        morphology/
                contain one or several morphologies (.swc)
                used for the model
        hall_of_fame.json ( contain the parameter sets - the results of the optimisation)
        val_models.json ( optional file, if several morphologies are used, the parameter sets which match each morphology)
        
For an example of the structure and contents of the files, see **BasalGangliaData/tests/test_data/example_variation_source**
                
       
                
    

The transfer has been divided into several steps.

Create directory for the model.
    
    Within BasalGangliaData, the models used in Snudda are saved under
    BasalGangliaData/data/neurons/name_of_nucleus

    **If the nucleus does not exist, add a folder for the new nucleus**

    Next create (if it does not already exist), a folder for each cell type within the nucleus

    Lastly, create the folder for each model of the cell type 
    (this folder will be the **destination** used in the code below)

     For example,

     BasalGanglia/data/neurons/newnucleus/new_celltype/new_model
        
    

### Add tools to your path

```
import sys
sys.path.append("../tools")
source = "where the Bluepyopt optimisation, with the structure described above"
destination = "BasalGanglia/data/neurons/newnucleus/new_celltype/new_model"
```

### Transfer mechanisms

```
from transfer.mechanisms import transfer_mechanisms
transfer_mechanisms(source=source, destination=destination)
```

### Transfer parameters

```
from transfer.parameters import transfer_parameters
transfer_parameters(source=source,
                            destination=destination,
                            selected=True)
```

### Transfer selected models from val_models.json

```
from transfer.selected_models import transfer_selected_models
transfer_selected_models(source=source, destination=destination)
```

### Transfer morphologies

```
from transfer.morphology import transfer_morphologies
transfer_morphologies(source=source,
                              destination=destination,
                              selected=True)
```

### Create the meta.json which combines all information on the model

```
from meta.create_meta import write_meta
write_meta(directory=destination, selected=True)
```